<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_PatchTST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ekaterineghibradze","key":"b1414052fbae86987efff2083c8dcbd1"}'}

In [5]:
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!ls -l ~/.kaggle/

total 4
-rw------- 1 root root 74 Jul  6 14:37 kaggle.json


In [7]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 896MB/s]


In [8]:
! unzip walmart-recruiting-store-sales-forecasting

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [9]:
!unzip '*.csv.zip'

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  features.csv.zip
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
  inflating: train.csv               

4 archives were successfully processed.


In [10]:
!unzip '*.csv.zip'

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               

4 archives were successfully processed.


In [11]:
import pandas as pd

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')

# Explore the data
print("Train shape:", train.shape)
print(train.head())
print(train.columns)

print("\nTest shape:", test.shape)
print(test.head())
print(test.columns)

print("\nFeatures shape:", features.shape)
print(features.head())
print(features.columns)

print("\nStores shape:", stores.shape)
print(stores.head())
print(stores.columns)

print("\nSample Submission shape:", sample_submission.shape)
print(sample_submission.head())
print(sample_submission.columns)

Train shape: (421570, 5)
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object')

Test shape: (115064, 4)
   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
Index(['Store', 'Dept', 'Date', 'IsHoliday'], dtype='object')

Features shape: (8190, 12)
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        3

In [12]:
# Merge train and test with features and stores
train_merged = pd.merge(train, features, on=['Store', 'Date'], how='left')
train_merged = pd.merge(train_merged, stores, on='Store', how='left')

test_merged = pd.merge(test, features, on=['Store', 'Date'], how='left')
test_merged = pd.merge(test_merged, stores, on='Store', how='left')

# Convert Date to datetime
train_merged['Date'] = pd.to_datetime(train_merged['Date'])


In [13]:
# Split
train_data = train_merged[train_merged['Date'] < '2012-01-01']
val_data = train_merged[(train_merged['Date'] >= '2012-01-01') & (train_merged['Date'] < '2012-07-01')]
test_data = train_merged[train_merged['Date'] >= '2012-07-01']

print("Train:", train_data.shape)
print("Validation:", val_data.shape)
print("Test (local):", test_data.shape)

Train: (294132, 17)
Validation: (77110, 17)
Test (local): (50328, 17)


In [14]:
import wandb
wandb.login()
wandb.init(project="Store-Sales-Forecasting", entity="agasi22-free-university-of-tbilisi-" , name="patchTST-training-run")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eghib22 (eghib22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [15]:
def preprocess(df):
    df = df.copy()

    # Encode 'Type' categorical
    if 'Type' in df.columns:
        type_map = {'A': 0, 'B': 1, 'C': 2}
        df['Type'] = df['Type'].map(type_map)

    # Handle IsHoliday columns robustly
    if 'IsHoliday_x' in df.columns and 'IsHoliday_y' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].fillna(0).astype(int)
        df = df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
    elif 'IsHoliday_x' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].fillna(0).astype(int)
        df = df.drop(columns=['IsHoliday_x'])
    elif 'IsHoliday_y' in df.columns:
        df['IsHoliday'] = df['IsHoliday_y'].fillna(0).astype(int)
        df = df.drop(columns=['IsHoliday_y'])
    elif 'IsHoliday' in df.columns:
        df['IsHoliday'] = df['IsHoliday'].fillna(0).astype(int)
    else:
        # No IsHoliday column at all — create one with zeros
        df['IsHoliday'] = 0

    # Ensure Date is datetime type
    if 'Date' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Extract date features only if Date column exists
    if 'Date' in df.columns:
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Week'] = df['Date'].dt.isocalendar().week
        df['Day'] = df['Date'].dt.day
        df = df.drop(columns=['Date'])

    # Fill MarkDown NaNs safely
    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    return df


# Preprocess train and val
train_data_processed = preprocess(train_data)
val_data_processed = preprocess(val_data)

# Example weights for WMAE:
weights_val = val_data_processed['IsHoliday'].apply(lambda x: 5 if x == 1 else 1)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import wandb
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

class PatchTST(nn.Module):
    def __init__(self, n_vars, patch_len, stride, d_model, n_heads, n_layers, d_ff, dropout=0.1):
        super(PatchTST, self).__init__()

        self.n_vars = n_vars
        self.patch_len = patch_len
        self.stride = stride
        self.d_model = d_model

        # Patch embedding layer
        self.patch_embedding = nn.Linear(patch_len, d_model)

        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.randn(1000, d_model))

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # Output projection
        self.output_projection = nn.Linear(d_model, 1)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def create_patches(self, x):
        # x shape: (batch_size, seq_len, n_vars)
        batch_size, seq_len, n_vars = x.shape

        # Create patches for each variable
        patches = []
        for i in range(n_vars):
            var_data = x[:, :, i]  # (batch_size, seq_len)
            var_patches = []

            # Create patches with stride
            for start in range(0, seq_len - self.patch_len + 1, self.stride):
                end = start + self.patch_len
                patch = var_data[:, start:end]  # (batch_size, patch_len)
                var_patches.append(patch)

            if var_patches:
                var_patches = torch.stack(var_patches, dim=1)  # (batch_size, n_patches, patch_len)
                patches.append(var_patches)

        if patches:
            patches = torch.cat(patches, dim=1)  # (batch_size, n_vars * n_patches, patch_len)
        else:
            # Fallback if no patches could be created
            patches = torch.zeros(batch_size, 1, self.patch_len, device=x.device)

        return patches

    def forward(self, x):
        # Create patches
        patches = self.create_patches(x)  # (batch_size, n_patches, patch_len)
        batch_size, n_patches, _ = patches.shape

        # Embed patches
        embedded = self.patch_embedding(patches)  # (batch_size, n_patches, d_model)

        # Add positional encoding
        pos_enc = self.positional_encoding[:n_patches, :].unsqueeze(0).expand(batch_size, -1, -1)
        embedded = embedded + pos_enc

        # Apply dropout
        embedded = self.dropout(embedded)

        # Pass through transformer
        transformer_out = self.transformer(embedded)  # (batch_size, n_patches, d_model)

        # Global average pooling
        pooled = transformer_out.mean(dim=1)  # (batch_size, d_model)

        # Output projection
        output = self.output_projection(pooled)  # (batch_size, 1)

        return output.squeeze(-1)  # (batch_size,)

def create_sequences(df, seq_len=52, target_col='Weekly_Sales'):
    """Create sequences for time series prediction"""
    sequences = []
    targets = []

    # Group by store to maintain temporal order within each store
    for store in df['Store'].unique():
        store_data = df[df['Store'] == store].sort_values('Year').reset_index(drop=True)

        if len(store_data) < seq_len + 1:
            continue

        # Select relevant features (excluding target and store ID)
        feature_cols = [col for col in store_data.columns if col not in ['Weekly_Sales', 'Store']]
        store_features = store_data[feature_cols].values
        store_targets = store_data[target_col].values

        # Create sequences
        for i in range(len(store_data) - seq_len):
            seq = store_features[i:i + seq_len]
            target = store_targets[i + seq_len]
            sequences.append(seq)
            targets.append(target)

    return np.array(sequences), np.array(targets)

def calculate_wmae(y_true, y_pred, weights):
    """Calculate Weighted Mean Absolute Error"""
    mae = np.abs(y_true - y_pred)
    wmae = np.sum(weights * mae) / np.sum(weights)
    return wmae

def train_model(model, train_loader, val_loader, num_epochs=50, lr=0.001, device='cpu'):
    """Train the PatchTST model"""
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

    best_val_loss = float('inf')

    # Watch model for wandb
    wandb.watch(model, log='all', log_freq=10)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        scheduler.step(avg_val_loss)

        # Log metrics to wandb
        wandb.log({
            'epoch': epoch,
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'learning_rate': optimizer.param_groups[0]['lr']
        })

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict().copy()

            # Log best validation loss
            wandb.log({'best_val_loss': best_val_loss})

        if epoch % 10 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    # Load best model
    model.load_state_dict(best_model_state)
    return model

def main():
    # Initialize wandb
    wandb.init(
        project="store-sales-prediction",
        name="PatchTST",
        config={
            "model": "PatchTST",
            "seq_len": 52,
            "batch_size": 32,
            "patch_len": 4,
            "stride": 2,
            "d_model": 64,
            "n_heads": 4,
            "n_layers": 2,
            "d_ff": 128,
            "dropout": 0.1,
            "learning_rate": 0.001,
            "num_epochs": 50,
            "optimizer": "Adam",
            "loss_function": "MSE",
            "scheduler": "ReduceLROnPlateau"
        }
    )

    # Load and preprocess data (using your existing code)
    print("Loading and preprocessing data...")

    # Your existing data loading and preprocessing code would go here
    # For this example, I'll assume the data is already loaded as per your code

    # Parameters
    seq_len = wandb.config.seq_len
    batch_size = wandb.config.batch_size
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Log device info
    wandb.log({"device": str(device)})

    # Create sequences
    print("Creating sequences...")
    X_train, y_train = create_sequences(train_data_processed, seq_len)
    X_val, y_val = create_sequences(val_data_processed, seq_len)

    print(f"Training sequences shape: {X_train.shape}")
    print(f"Validation sequences shape: {X_val.shape}")

    # Log data information
    wandb.log({
        "train_samples": len(X_train),
        "val_samples": len(X_val),
        "n_features": X_train.shape[-1],
        "sequence_length": X_train.shape[1]
    })

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Scale targets
    target_scaler = StandardScaler()
    y_train_scaled = target_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_val_scaled = target_scaler.transform(y_val.reshape(-1, 1)).flatten()

    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    y_train_tensor = torch.FloatTensor(y_train_scaled)
    X_val_tensor = torch.FloatTensor(X_val_scaled)
    y_val_tensor = torch.FloatTensor(y_val_scaled)

    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    n_vars = X_train.shape[-1]  # Number of features
    model = PatchTST(
        n_vars=n_vars,
        patch_len=wandb.config.patch_len,
        stride=wandb.config.stride,
        d_model=wandb.config.d_model,
        n_heads=wandb.config.n_heads,
        n_layers=wandb.config.n_layers,
        d_ff=wandb.config.d_ff,
        dropout=wandb.config.dropout
    )

    num_params = sum(p.numel() for p in model.parameters())
    print(f"Model initialized with {num_params} parameters")

    # Log model information
    wandb.log({"model_parameters": num_params})

    # Train model
    print("Training model...")
    model = train_model(model, train_loader, val_loader,
                       num_epochs=wandb.config.num_epochs,
                       lr=wandb.config.learning_rate,
                       device=device)

    # Make predictions
    print("Making predictions...")
    model.eval()

    with torch.no_grad():
        # Training predictions
        train_preds = []
        for batch_x, _ in train_loader:
            batch_x = batch_x.to(device)
            pred = model(batch_x)
            train_preds.append(pred.cpu().numpy())
        train_preds = np.concatenate(train_preds)

        # Validation predictions
        val_preds = []
        for batch_x, _ in val_loader:
            batch_x = batch_x.to(device)
            pred = model(batch_x)
            val_preds.append(pred.cpu().numpy())
        val_preds = np.concatenate(val_preds)

    # Inverse transform predictions
    train_preds_original = target_scaler.inverse_transform(train_preds.reshape(-1, 1)).flatten()
    val_preds_original = target_scaler.inverse_transform(val_preds.reshape(-1, 1)).flatten()

    # Calculate weights for WMAE
    # For training data, create weights based on IsHoliday
    train_weights = np.ones(len(y_train))  # Default weight of 1
    val_weights = np.ones(len(y_val))      # Default weight of 1

    # You would need to align these weights with your actual holiday data
    # For now, using equal weights as an example

    # Calculate WMAE
    train_wmae = calculate_wmae(y_train, train_preds_original, train_weights)
    val_wmae = calculate_wmae(y_val, val_preds_original, val_weights)

    # Calculate additional metrics
    train_mae = mean_absolute_error(y_train, train_preds_original)
    val_mae = mean_absolute_error(y_val, val_preds_original)

    # Log final metrics
    wandb.log({
        "final_train_wmae": train_wmae,
        "final_val_wmae": val_wmae,
        "final_train_mae": train_mae,
        "final_val_mae": val_mae
    })

    # Create prediction vs actual plots
    import matplotlib.pyplot as plt

    # Training predictions plot
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.scatter(y_train[:1000], train_preds_original[:1000], alpha=0.5)
    plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
    plt.xlabel('Actual Sales')
    plt.ylabel('Predicted Sales')
    plt.title('Training: Predicted vs Actual')

    # Validation predictions plot
    plt.subplot(2, 2, 2)
    plt.scatter(y_val[:1000], val_preds_original[:1000], alpha=0.5)
    plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
    plt.xlabel('Actual Sales')
    plt.ylabel('Predicted Sales')
    plt.title('Validation: Predicted vs Actual')

    # Training residuals
    plt.subplot(2, 2, 3)
    train_residuals = y_train[:1000] - train_preds_original[:1000]
    plt.scatter(train_preds_original[:1000], train_residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Sales')
    plt.ylabel('Residuals')
    plt.title('Training Residuals')

    # Validation residuals
    plt.subplot(2, 2, 4)
    val_residuals = y_val[:1000] - val_preds_original[:1000]
    plt.scatter(val_preds_original[:1000], val_residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Sales')
    plt.ylabel('Residuals')
    plt.title('Validation Residuals')

    plt.tight_layout()

    # Log the plot to wandb
    wandb.log({"prediction_plots": wandb.Image(plt)})
    plt.close()

    # Create a summary table
    summary_data = [
        ["Training WMAE", train_wmae],
        ["Validation WMAE", val_wmae],
        ["Training MAE", train_mae],
        ["Validation MAE", val_mae],
        ["Model Parameters", num_params],
        ["Training Samples", len(X_train)],
        ["Validation Samples", len(X_val)]
    ]

    table = wandb.Table(data=summary_data, columns=["Metric", "Value"])
    wandb.log({"results_summary": table})

    print(f"\nResults:")
    print(f"Training WMAE: {train_wmae:.4f}")
    print(f"Validation WMAE: {val_wmae:.4f}")
    print(f"Training MAE: {train_mae:.4f}")
    print(f"Validation MAE: {val_mae:.4f}")

    # Save model artifact
    model_artifact = wandb.Artifact("patchtst_model", type="model")
    torch.save(model.state_dict(), "patchtst_model.pth")
    model_artifact.add_file("patchtst_model.pth")
    wandb.log_artifact(model_artifact)

    # Finish wandb run
    wandb.finish()

    return model, train_wmae, val_wmae

if __name__ == "__main__":
    model, train_wmae, val_wmae = main()

Loading and preprocessing data...
Creating sequences...
Training sequences shape: (291792, 52, 17)
Validation sequences shape: (74770, 52, 17)
Model initialized with 131329 parameters
Training model...
Epoch [0/50], Train Loss: 0.8019, Val Loss: 0.7280
